<img src="https://www.ebi.ac.uk/ena/browser/assets/ENA_logo_2021.png" width="350" style="float: right;"> 
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/9/94/Coronavirus._SARS-CoV-2.png/220px-Coronavirus._SARS-CoV-2.png" widtf="150" style="float: right;">

# SQL queries to Biological databases 

## Analysis of SARS-COV-2 mutation database

This notebook explores the [SARS-Cov-2](https://en.wikipedia.org/wiki/Severe_acute_respiratory_syndrome_coronavirus_2) VCF database. The database consists of 4 tables with data of [ENA (European Nucleotide Archive)](https://www.ebi.ac.uk/ena/browser/home) sequencing runs. [VCF](https://en.wikipedia.org/wiki/Variant_Call_Format) datasets store information about sequence variants, in our case divergent types of Sars-Cov2 virus sequences. An extensive metadata table is also available, which can be linked to VCF data via the ENA run ID. Lineage tables contain information about the types and number of mutations that define different variants.

### 0. Connect to the production or replica databases

The picture below shows the diagram of the database (only the relevant part). The lines show which table uses foreign keys from which other table. The table `lineage` is a materialized view and no foreign key can be explicitly declared there. 

![](./db_diagramm.png)

In [ ]:
# import required libraries

import psycopg2 # https://anaconda.org/anaconda/psycopg2
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
import time
import json

# Hide warnings for now
import warnings
warnings.filterwarnings('ignore')

Connection parameters

In [ ]:
conn = {}
with open("/v/courses/dataesp2023.public/Datasets/D-SQL/coveo_connection.json", 'r') as f:
    conn = json.load(f)

# Choose a server
server = conn['ELTE_replica']

In [ ]:
# A function for submitting our queries
# It opens a connection to the database, receives the result and closes the connection
def execute_query(query, server):
    start_time = time.time()
    try:
        with psycopg2.connect(f"host='{server['host']}' port={server['port']} dbname='{server['dbname']}' user={server['user']} \
                              password={server['password']} options='{server['options']}'") as conn:
            data = pd.read_sql_query(query, conn)
    except Exception as e:
        print("Something went wrong.", e)

    elapsed_time = time.time() - start_time
    minutes, seconds = divmod(elapsed_time, 60)
    hours, minutes = divmod(minutes, 60)
    print("\033[32mRuntime: {:.0f} hours, {:.0f} minutes, {:.2f} seconds\033[0m".format(hours, minutes, seconds))
    
    return data

    i) Get the list of tables in the Coveo database, under the `datahub_0` scheme! (1p)

### 1. Simple analysis of the tables

    i) Take a look at the head of the `metadata`, `vcf_key`, `vcf`, `cov` and `country` tables (1p)

    ii) What is the earliest and last collection date os the samples? (Max and Min) (1p)

    iii) Number of all samples in `metadata` (1p)

    iv) Number of all samples in `vcf` (1p)

### 2. Further analysis

    i) Are there samples that is only in the `metadata` or only in `vcf` tables? (2p)
    Possible reason: no mutation in the sample, subsequent submit, manual error

    ii) Where do the samples come from? (2p)

### 3. Search for mutations of interest in the database

    i) How many samples (runid) in the database contain the following mutations? (use vcf table): a T9I amino acid mutation in the coat protein, caused by a cytosine (vcf.ref=C) - thymine (vcf.alt=T) change at position 26270 (pos = 26270) in the genome (2p)
    

    ii) - How many samples remain if only those with more definite mutations are tested (qualitative filter: min 90% allele frequency (vcf.af > 0.9), min 100 coverage (vcf.dp > 100))? (use `vcf` table) (2p)

    iii) In which countries (country.id) have these mutations been detected? (use `vcf`, `metadata`, `country` tables) (3p)

    iv) In which variants (lineage.variant_id) are these mutations common? (use `vcf`, `metadata`, `lineage` tables) (3p)

### 4. In which variants (variant_id) contain the most deletion (LENGTH(vcf.ref) > LENGTH(vcf.alt)) or insertion (LENGTH(vcf.ref) < LENGTH(vcf.alt)) (use the `lineage` and`vcf` tables)? (4p)